In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table align="left">
  <td>
    <a href="https://github.com/GoogleCloudPlatform/ai-platform-samples/blob/master/ai-platform-unified/notebooks/notebook_template.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
</table>

## Overview

This tutorial walks through building a custom container to serve a scikit-learn model on AI Platform Predictions. You will use the FastAPI Python web server framework to create a prediction and health endpoint.
You will also cover incorporating a pre-processor from training into your online serving.


### Dataset

This tutorial uses R.A. Fisher's Iris dataset, a small dataset that is popular for trying out machine learning techniques. Each instance has four numerical features, which are different measurements of a flower, and a target label that
marks it as one of three types of iris: Iris setosa, Iris versicolour, or Iris virginica.

This tutorial uses [the copy of the Iris dataset included in the
scikit-learn library](https://scikit-learn.org/stable/datasets/index.html#iris-dataset).

### Objective

The goal is to:
- Train a model that uses a flower's measurements as input to predict what type of iris it is.
- Save the model and its serialized pre-processor
- Build a FastAPI server to handle predictions and health checks
- Build a custom container with model artifacts
- Upload and deploy custom container to AI Platform Prediction

This tutorial focuses more on deploying this model with AI Platform than on
the design of the model itself.

### Costs 

This tutorial uses billable components of Google Cloud:

* AI Platform (Unified)

Learn about [AI Platform (Unified)
pricing](https://cloud.google.com/ai-platform-unified/pricing), and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

### Set up your local development environment

**If you are using Colab or AI Platform Notebooks**, your environment already meets
all the requirements to run this notebook. You can skip this step.

**Otherwise**, make sure your environment meets this notebook's requirements.
You need the following:

* Docker
* Git
* Google Cloud SDK (gcloud)
* Python 3
* virtualenv
* Jupyter notebook running in a virtual environment with Python 3

The Google Cloud guide to [Setting up a Python development
environment](https://cloud.google.com/python/setup) and the [Jupyter
installation guide](https://jupyter.org/install) provide detailed instructions
for meeting these requirements. The following steps provide a condensed set of
instructions:

1. [Install and initialize the Cloud SDK.](https://cloud.google.com/sdk/docs/)

1. [Install Python 3.](https://cloud.google.com/python/setup#installing_python)

1. [Install
   virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv)
   and create a virtual environment that uses Python 3. Activate the virtual environment.

1. To install Jupyter, run `pip install jupyter` on the
command-line in a terminal shell.

1. To launch Jupyter, run `jupyter notebook` on the command-line in a terminal shell.

1. Open this notebook in the Jupyter Notebook Dashboard.

### Install additional packages

Install additional package dependencies not installed in your notebook environment, such as NumPy, Scikit-learn, FastAPI, Uvicorn, and joblib. Use the latest major GA version of each package.

In [ ]:
%%writefile requirements.txt
joblib~=1.0
numpy~=1.20
scikit-learn~=0.24

In [ ]:
# Required in Docker serving container
%pip install -U -r requirements.txt --user

# For local FastAPI development and running
%pip install -U "uvicorn[standard]>=0.12.0,<0.14.0" fastapi~=0.63 --user

# AI Platform (Unified) SDK
%pip install -U google-cloud-aiplatform --user

### Restart the kernel

After you install the additional packages, you need to restart the notebook kernel so it can find the packages.

In [ ]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## Before you begin

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

1. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

1. [Enable the AI Platform (Unified) API and Compute Engine API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component). {TODO: Update the APIs needed for your tutorial. Edit the API names, and update the link to append the API IDs, separating each one with a comma. For example, container.googleapis.com,cloudbuild.googleapis.com}

1. If you are running this notebook locally, you will need to install the [Cloud SDK](https://cloud.google.com/sdk).

1. Enter your project ID in the cell below. Then run the cell to make sure the
Cloud SDK uses the right project for all the commands in this notebook.

**Note**: Jupyter runs lines prefixed with `!` as shell commands, and it interpolates Python variables prefixed with `$` into these commands.

#### Set your project ID

**If you don't know your project ID**, you may be able to get your project ID using `gcloud`.

In [ ]:
# Get your Google Cloud project ID from gcloud
shell_output=!gcloud config list --format 'value(core.project)' 2>/dev/null
PROJECT = shell_output[0]
print("Project ID: ", PROJECT)

Otherwise, set your project ID here.

In [ ]:
if PROJECT == "" or PROJECT is None:
    PROJECT = "PROJECT_ID"  # @param {type:"string"}

### Authenticate your Google Cloud account

**If you are using AI Platform Notebooks**, your environment is already
authenticated. Skip this step.

**If you are using Colab**, run the cell below and follow the instructions
when prompted to authenticate your account via oAuth.

**Otherwise**, follow these steps:

1. In the Cloud Console, go to the [**Create service account key**
   page](https://console.cloud.google.com/apis/credentials/serviceaccountkey).

2. Click **Create service account**.

3. In the **Service account name** field, enter a name, and
   click **Create**.

4. In the **Grant this service account access to project** section, click the **Role** drop-down list. Type "AI Platform"
into the filter box, and select
   **AI Platform Administrator**. Type "Storage Object Admin" into the filter box, and select **Storage Object Admin**.

5. Click *Create*. A JSON file that contains your key downloads to your
local environment.

6. Enter the path to your service account key as the
`GOOGLE_APPLICATION_CREDENTIALS` variable in the cell below and run the cell.

In [ ]:
import os
import sys

# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# If on AI Platform, then don't execute this code
if not os.path.exists("/opt/deeplearning/metadata/env_version"):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING") and not os.getenv("GOOGLE_APPLICATION_CREDENTIALS"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### Configure project and naming

In [ ]:
REGION = "us-central1"  # @param {type:"string"}
REPOSITORY = "custom-container-prediction-sklearn"  # @param {type:"string"}
IMAGE = "sklearn-fastapi-server"  # @param {type:"string"}
MODEL_DISPLAY_NAME = "sklearn_custom_container"  # @param {type:"string"}

In [ ]:
%mkdir app

## Write your pre-processor
Scaling training data so each numerical feature column has a mean of 0 and a standard deviation of 1 [can improve your model](https://developers.google.com/machine-learning/crash-course/representation/cleaning-data).

Create `preprocess.py`, which contains a class to do this scaling:

In [ ]:
%%writefile app/preprocess.py
import numpy as np

class MySimpleScaler(object):
    def __init__(self):
        self._means = None
        self._stds = None

    def preprocess(self, data):
        if self._means is None:  # during training only
            self._means = np.mean(data, axis=0)

        if self._stds is None:  # during training only
            self._stds = np.std(data, axis=0)
            if not self._stds.all():
                raise ValueError("At least one column has standard deviation of 0.")

        return (data - self._means) / self._stds


## Train and store model with pre-processor
Next, use `preprocess.MySimpleScaler` to preprocess the iris data, then train a model using scikit-learn.

At the end, export your trained model as a joblib (`.joblib`) file and export your `MySimpleScaler` instance as a pickle (`.pkl`) file:

In [ ]:
%cd app/

import joblib
import pickle

from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
from preprocess import MySimpleScaler

iris = load_iris()
scaler = MySimpleScaler()

X = scaler.preprocess(iris.data)
y = iris.target

model = RandomForestClassifier()
model.fit(X, y)

joblib.dump(model, "model.joblib")
with open ("preprocessor.pkl", "wb") as f:
  pickle.dump(scaler, f)

%cd ..

## Build a FastAPI server

In [ ]:
%%writefile app/main.py
from fastapi import FastAPI, Request

import pickle
import joblib
import json
import numpy as np

from sklearn.datasets import load_iris
from preprocess import MySimpleScaler

app = FastAPI()

with open("preprocessor.pkl", "rb") as f:
    preprocessor = pickle.load(f)

_class_names = load_iris().target_names
_model = joblib.load("model.joblib")
_preprocessor = preprocessor


@app.get("/health", status_code=200)
def health():
    return {}


@app.post("/predict")
async def predict(request: Request):
    body = await request.json()

    instances = body["instances"]
    inputs = np.asarray(instances)
    preprocessed_inputs = _preprocessor.preprocess(inputs)
    outputs = _model.predict(preprocessed_inputs)

    return {"predictions": [_class_names[class_num] for class_num in outputs]}


### Store test instances to use later
To learn more about formatting input instances in JSON, [read the documentation.](https://cloud.google.com/ai-platform-unified/docs/predictions/online-predictions-custom-models#request-body-details)

In [ ]:
%%writefile instances.json
{
    "instances": [
        [6.7, 3.1, 4.7, 1.5],
        [4.6, 3.1, 1.5, 0.2]
    ]
}

## Build and push container to Artifact Registry

### Build your container

In [ ]:
%%writefile Dockerfile

FROM tiangolo/uvicorn-gunicorn-fastapi:python3.7

COPY ./app /app
COPY requirements.txt requirements.txt

RUN pip install -r requirements.txt

In [ ]:
!docker build \
    --tag={REGION}-docker.pkg.dev/{PROJECT}/{REPOSITORY}/{IMAGE} \
    .

### Run and test the container locally (optional)

Run the container locally in detached mode and test the `/health` and `/predict` routes. Afterwards, stop the running image.

In [ ]:
!docker run -d -p 80:80 \
    --name=local-iris \
    {REGION}-docker.pkg.dev/{PROJECT}/{REPOSITORY}/{IMAGE}

In [ ]:
!curl localhost/health

In [ ]:
!curl -X POST \
  -d @instances.json \
  -H "Content-Type: application/json; charset=utf-8" \
  localhost/predict

In [ ]:
!docker stop local-iris

### Push the container to artifact registry

Configure Docker to access Artifact Registry. Then push your container image to your Artifact Registry repository.

In [ ]:
!gcloud beta artifacts repositories create {REPOSITORY} \
 --repository-format=docker \
 --location=$REGION

In [ ]:
!gcloud auth configure-docker {REGION}-docker.pkg.dev

In [ ]:
!docker push {REGION}-docker.pkg.dev/{PROJECT}/{REPOSITORY}/{IMAGE}

## Deploy to AI Platform (Unified)

Use the Python SDK to upload and deploy your model.

### Upload the custom container model

In [ ]:
from google.cloud import aiplatform

In [ ]:
aiplatform.init(project=PROJECT, location=REGION)

In [ ]:
model = aiplatform.Model.upload(
  display_name=MODEL_DISPLAY_NAME,
  serving_container_image_uri=f"{REGION}-docker.pkg.dev/{PROJECT}/{REPOSITORY}/{IMAGE}",
  serving_container_ports=[80],
  serving_container_health_route="/health",
  serving_container_predict_route="/predict",
)

### Deploy the model on AI Platform (Unified)

In [ ]:
endpoint = model.deploy(machine_type="n1-standard-4")

## Send predictions

### Using Python SDK

In [ ]:
endpoint.predict(
    instances=[
        [6.7, 3.1, 4.7, 1.5],
        [4.6, 3.1, 1.5, 0.2]
    ]
)

### Using REST

In [ ]:
ENDPOINT_ID = endpoint.name

In [ ]:
!curl \
    -H "Authorization: Bearer $(gcloud auth print-access-token)" \
    -H "Content-Type: application/json" \
    -d @instances.json \
    https://{REGION}-aiplatform.googleapis.com/v1/projects/{PROJECT}/locations/{REGION}/endpoints/{ENDPOINT_ID}:predict

### Using gcloud CLI

In [ ]:
!gcloud beta ai endpoints predict $ENDPOINT_ID \
  --region=$REGION \
  --json-request=instances.json

## Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial:

In [ ]:
# Undeploy model and delete endpoint
endpoint.delete(force=True)

# Delete the model resource
model.delete()

# Delete the container image from Artifact Registry
!gcloud artifacts docker images delete \
    --quiet \
    --delete-tags \
    {REGION}-docker.pkg.dev/{PROJECT}/{REPOSITORY}/{IMAGE}